In [1]:
import findspark
findspark.init()

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
spark = SparkSession.builder.appName('i1').getOrCreate()
spark

In [28]:
d1 = [('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3')]
c1 = ['customer_id', 'order_date','product_id']

df1 = spark.createDataFrame(data = d1, schema=c1)
df1.show()
df1.printSchema()

+-----------+----------+----------+
|customer_id|order_date|product_id|
+-----------+----------+----------+
|          A|2021-01-01|         1|
|          A|2021-01-01|         2|
|          A|2021-01-07|         2|
|          A|2021-01-10|         3|
|          A|2021-01-11|         3|
|          A|2021-01-11|         3|
|          B|2021-01-01|         2|
|          B|2021-01-02|         2|
|          B|2021-01-04|         1|
|          B|2021-01-11|         1|
|          B|2021-01-16|         3|
|          B|2021-02-01|         3|
|          C|2021-01-01|         3|
|          C|2021-01-01|         3|
|          C|2021-01-07|         3|
+-----------+----------+----------+

root
 |-- customer_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- product_id: string (nullable = true)



In [29]:
# read and change dtypes

sales = df1.withColumn('order_date', col('order_date').cast(DateType())).withColumn('product_id', df1.product_id.cast(IntegerType()))
sales.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- product_id: integer (nullable = true)



In [26]:
d2 = [('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12')]
c2 = ['product_id', 'product_name', 'price']
df2 = spark.createDataFrame(data = d2, schema=c2)
df2.show()
df2.printSchema()

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         1|       sushi|   10|
|         2|       curry|   15|
|         3|       ramen|   12|
+----------+------------+-----+

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: string (nullable = true)



In [32]:
menu = df2.select(df2.product_id.cast(IntegerType()), col('price').cast(IntegerType()), 'product_name')
menu.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- product_name: string (nullable = true)



In [33]:
d3=[('A', '2021-01-07'),
  ('B', '2021-01-09')]
c3 = ['customer_id','join_date']
df3 = spark.createDataFrame(data=d3, schema=c3)
df3.show()
df3.printSchema()

+-----------+----------+
|customer_id| join_date|
+-----------+----------+
|          A|2021-01-07|
|          B|2021-01-09|
+-----------+----------+

root
 |-- customer_id: string (nullable = true)
 |-- join_date: string (nullable = true)



In [37]:
members = df3.select('customer_id',df3.join_date.cast(DateType()))
members.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- join_date: date (nullable = true)



In [38]:
sales.show()
menu.show()
members.show()

+-----------+----------+----------+
|customer_id|order_date|product_id|
+-----------+----------+----------+
|          A|2021-01-01|         1|
|          A|2021-01-01|         2|
|          A|2021-01-07|         2|
|          A|2021-01-10|         3|
|          A|2021-01-11|         3|
|          A|2021-01-11|         3|
|          B|2021-01-01|         2|
|          B|2021-01-02|         2|
|          B|2021-01-04|         1|
|          B|2021-01-11|         1|
|          B|2021-01-16|         3|
|          B|2021-02-01|         3|
|          C|2021-01-01|         3|
|          C|2021-01-01|         3|
|          C|2021-01-07|         3|
+-----------+----------+----------+

+----------+-----+------------+
|product_id|price|product_name|
+----------+-----+------------+
|         1|   10|       sushi|
|         2|   15|       curry|
|         3|   12|       ramen|
+----------+-----+------------+

+-----------+----------+
|customer_id| join_date|
+-----------+----------+
|          A|20

In [41]:
# What is the total amount each customer spent at the restaurant

totalamtdf = (sales.join(menu, 'product_id')
                   .groupBy('customer_id').agg(sum('price'))
                   .withColumnRenamed('sum(price)', 'totalamount')
                   .orderBy('customer_id') 
             )
totalamtdf.show()

+-----------+-----------+
|customer_id|totalamount|
+-----------+-----------+
|          A|         76|
|          B|         74|
|          C|         36|
+-----------+-----------+



In [ ]:
# How many days has each customer visited the restau

In [42]:
spark.stop()